In [13]:
import pandas as pd
import os
import subprocess
import shutil

In [14]:
metadata_path = '/neurospin/micromri/Georges/rats_white_matter/data/multirat_se/script/table/metadata_stand.tsv'
dataset_path = '/neurospin/micromri/Georges/rats_white_matter/data/ds005534'
preprocessed_path = '/neurospin/micromri/Georges/rats_white_matter/data/preprocessed'

In [15]:
df = pd.read_csv(metadata_path, sep='\t')
df['MRI.field.strength'] = pd.to_numeric(df['MRI.field.strength'], errors='coerce')
target_strength = 17.2
selected_rats = df[df['MRI.field.strength'] == target_strength]
print(f"Found {len(selected_rats)} rats with MRI field strength {target_strength}.")

Found 20 rats with MRI field strength 17.2.


In [16]:
os.makedirs(preprocessed_path, exist_ok=True)

In [ ]:
for _, row in selected_rats.iterrows():
    rat_id = str(row['rat.sub'])
    subject = f"sub-0{rat_id}"
    if rat_id.startswith('sub-'): subject = rat_id 
    src_subject_path = os.path.join(dataset_path, subject)
    dst_subject_path = os.path.join(preprocessed_path, subject)
    if not os.path.exists(src_subject_path):
        print(f"  [ ] Skipping {subject}: Not found in ds005534.")
        continue
    print(f"\nProcessing {subject}...")
    rel_anat = os.path.join(subject, 'ses-1', 'anat', '*.nii.gz')
    rel_func = os.path.join(subject, 'ses-1', 'func', '*.nii.gz')
    
    print(f"   > Fetching data from OpenNeuro...")
    
    cmd = f"datalad get {rel_anat} {rel_func}"
    
    try:
        subprocess.run(cmd, shell=True, cwd=dataset_path, check=True)
    except subprocess.CalledProcessError:
        print(f"   ! Warning: Datalad get failed for {subject}. Files might be missing or network down.")
    if os.path.exists(dst_subject_path):
        print(f"   > Destination exists, skipping copy.")
    else:
        print(f"   > Copying to preprocessed...")
        try:
            subprocess.run(['cp', '-rL', src_subject_path, dst_subject_path], check=True)
            print(f"   > Success.")
        except subprocess.CalledProcessError as e:
            print(f"   ! Copy failed: {e}")
print("\n--------------------------------")
print("Pipeline complete.")